In [ ]:
import os
import base64
import cv2
import pytesseract
import numpy as np
from google.oauth2 import service_account
from googleapiclient.discovery import build

# 🔹 Gmail API Setup
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
SERVICE_ACCOUNT_FILE = 'credentials.json'  # Replace with your credentials file
SAVE_FOLDER = "attachments"  # Folder to store images

def get_gmail_service():
    """ Authenticate and return the Gmail service """
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    service = build('gmail', 'v1', credentials=creds)
    return service

def fetch_screentime_email():
    """ Fetch emails from 'Screentime' or 'Digital Wellbeing' containing images """
    service = get_gmail_service()
    query = "from:(Screentime) OR from:(Digital Wellbeing)"  # Search sender
    results = service.users().messages().list(userId='me', q=query, maxResults=5).execute()
    messages = results.get('messages', [])

    if not messages:
        print("No relevant emails found.")
        return None

    os.makedirs(SAVE_FOLDER, exist_ok=True)

    for message in messages:
        msg = service.users().messages().get(userId='me', id=message['id']).execute()
        payload = msg['payload']
        
        for part in payload.get('parts', []):
            if part['filename'] and 'image' in part['mimeType']:
                attachment_id = part['body']['attachmentId']
                attachment = service.users().messages().attachments().get(
                    userId='me', messageId=message['id'], id=attachment_id).execute()

                data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
                file_path = os.path.join(SAVE_FOLDER, part['filename'])

                with open(file_path, "wb") as f:
                    f.write(data)
                print(f"Saved Image: {file_path}")
                return file_path  # Process only first relevant image
    return None

def extract_text_from_graph(image_path):
    """ Extracts screen time data using OCR """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    
    print("Extracted Text from Image:\n", text)
    return text

def extract_bar_graph_data(image_path):
    """ Extracts screen time values from bar graph using OpenCV """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bar_heights = [cv2.boundingRect(cnt)[3] for cnt in contours]  # Extract heights
    bar_heights.sort(reverse=True)

    print("Extracted Bar Heights:", bar_heights)
    return bar_heights

def store_screen_time(text):
    """ Check for Netflix, Spotify, and Swiggy and store screen time """
    relevant_apps = ["Netflix", "Spotify", "Swiggy"]
    with open("screen_time_log.txt", "a") as log:
        for app in relevant_apps:
            if app.lower() in text.lower():
                log.write(f"{app} Screen Time: {text}\n")
                print(f"Stored {app} screen time!")

# 🔥 Main Execution
image_path = fetch_screentime_email()
if image_path:
    extracted_text = extract_text_from_graph(image_path)  # OCR
    extracted_bars = extract_bar_graph_data(image_path)  # Bar graph analysis
    store_screen_time(extracted_text)
